In [7]:
###Important click through this to clean the data and get the instruction data before running the other notebooks###


In [8]:
#Imports
import os
import shutil
import random
import json
import re
import ast

In [9]:
# Paths please change these to your own paths
feed = r"C:\Users\elias\Documents\GitHub\BA\data\input\feed"
feed_pictures = r"C:\Users\elias\Documents\GitHub\BA\data\input\feedPictures"
output = r"C:\Users\elias\Documents\GitHub\BA\data\output"
path_img = r"E:\uni\BA\data\input\img"
path_box = r"E:\uni\BA\data\input\box"


add all the entities into the feed folder and all the pictures into the feed_pictures folder

In [10]:
def clean_price(price):
  # Remove any non-numeric characters except the decimal point
  cleaned_price = ''.join(char for char in price if char.isdigit() or char == '.')
    
  # Ensure it has two decimal places
  if '.' in cleaned_price:
    integer_part, decimal_part = cleaned_price.split('.')
    decimal_part = decimal_part[:2].ljust(2, '0')  # Ensure two decimal places
    corrected_price = f"{integer_part}.{decimal_part}"
  else:
    corrected_price = f"{cleaned_price}.00"  # No decimal point, add ".00"
    
  return corrected_price

In [11]:
#This function is used to clean the prices. Move the entities you want to have cleaned into your feed folder
def correct_price_format():

  folder_path = feed
  # Iterate over all files in the specified folder
  for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    with open(file_path, 'r') as file:
      data = json.load(file)
            
    # Check if the 'total' field is formatted correctly
    if 'total' in data:
      original_price = data['total']
      corrected_price = clean_price(original_price)

      if original_price != corrected_price:
        print(f"Correcting price in file {filename}: {original_price} -> {corrected_price}")
        data['total'] = corrected_price

                    # Write the corrected data back to the file
    with open(file_path, 'w') as file:
      json.dump(data, file, indent=4)
correct_price_format()

Correcting price in file X51005433492.txt: $8.20 -> 8.20
Correcting price in file X51005433514.txt: $8.20 -> 8.20
Correcting price in file X51005433518.txt: $8.20 -> 8.20
Correcting price in file X51005433538.txt: $8.20 -> 8.20
Correcting price in file X51005433541.txt: $8.20 -> 8.20
Correcting price in file X51005433543.txt: $8.20 -> 8.20
Correcting price in file X51005442327.txt: $8.60 -> 8.60
Correcting price in file X51005442333.txt: $6.90 -> 6.90
Correcting price in file X51005442334.txt: $8.20 -> 8.20
Correcting price in file X51005442375.txt: $8.50 -> 8.50
Correcting price in file X51005442376.txt: $7.60 -> 7.60
Correcting price in file X51005442379.txt: $9.20 -> 9.20
Correcting price in file X51005442382.txt: $6.60 -> 6.60
Correcting price in file X51005442384.txt: $8.20 -> 8.20
Correcting price in file X51005442386.txt: $8.70 -> 8.70
Correcting price in file X51005442392.txt: $7.10 -> 7.10
Correcting price in file X51005442394.txt: $8.20 -> 8.20
Correcting price in file X51005

LATIN helper

In [ ]:
def to_prompt(scan, img_size) -> str:
    # Convert ocr bboxes to latin boxes
    w, h = img_size
    texts = [x['text'] for x in scan]
    boxes = [x['bbox'] for x in scan]
      
    
    # Now continue with the latin prompting from https://github.dev/WenjinW/LATIN-Prompt
    line_boxes = []
    line_texts = []
    max_line_char_num = 0
    line_width = 0

    while len(boxes) > 0:
        line_box = [boxes.pop(0)]
        line_text = [texts.pop(0)]
        char_num = len(line_text[-1])
        line_union_box = line_box[-1]
        while len(boxes) > 0 and _is_same_line(line_box[-1], boxes[0]):
            line_box.append(boxes.pop(0))
            line_text.append(texts.pop(0))
            char_num += len(line_text[-1])
            line_union_box = _union_box(line_union_box, line_box[-1])
        line_boxes.append(line_box)
        line_texts.append(line_text)
        if char_num >= max_line_char_num:
            max_line_char_num = char_num
            line_width = line_union_box[2] - line_union_box[0]

    max_line_char_num = max(max_line_char_num, 1)
    char_width = line_width / max_line_char_num
    if char_width == 0:
        char_width = 1

    space_line_texts = []
    for i, line_box in enumerate(line_boxes):
        space_line_text = ""
        for j, box in enumerate(line_box):
            left_char_num = int(box[0] / char_width)
            space_line_text += " " * (left_char_num - len(space_line_text))
            space_line_text += line_texts[i][j]
        space_line_texts.append(space_line_text)

    return "\n".join(space_line_texts)


def _is_same_line(box1, box2):
    """
    Params:
        box1: [x1, y1, x2, y2]
        box2: [x1, y1, x2, y2]
    """
    box1_midy = (box1[1] + box1[3]) / 2
    box2_midy = (box2[1] + box2[3]) / 2

    if (
        box1_midy < box2[3]
        and box1_midy > box2[1]
        and box2_midy < box1[3]
        and box2_midy > box1[1]
    ):
        return True
    else:
        return False


def _union_box(box1, box2):
    """
    Params:
        box1: [x1, y1, x2, y2]
        box2: [x1, y1, x2, y2]
    """
    x1 = min(box1[0], box2[0])
    y1 = min(box1[1], box2[1])
    x2 = max(box1[2], box2[2])
    y2 = max(box1[3], box2[3])

    return [x1, y1, x2, y2]

Make sure, that the output folder is empty, before running.

In [ ]:
def latin_runner():
    
    image_files = [os.path.join(path_img, file) for file in os.listdir(path_img) if file.endswith(".jpg") or file.endswith(".png")]
    box_files = [os.path.join(path_box, file) for file in os.listdir(path_box) if file.endswith(".txt")]
    count = 1
    for image_file, box_file in zip(image_files, box_files):
        
        image = Image.open(image_file)
        box = open(box_file, "r")
        box.FileName = os.path.basename(box_file)
        print(box.FileName)
        box_lines = box.readlines()
        parsed_boxes = []
        for line in box_lines:
            line = line.strip()
            bbox, text  = Latin.split_after_eighth_comma(line,box.FileName)
            parsed_boxes.append({"text": text, "bbox": bbox})

     
        # For example, you can access its properties like width and height
        width, height = image.size
        img_size = (width, height)
        
        # Once you are done with the image, don't forget to close it
        full_output_path = os.path.join(output, box.FileName)
    
        with open(full_output_path, 'w') as output_file:
            output_file.write(Latin.to_prompt(parsed_boxes, img_size))
            print(count)
        count+=1
        image.close()
        box.close()
latin_runner()

Remove the LATIN layouts from the output folder and safe for future use.
Move the cleaned prices from the feed into another folder, they will later be used as the ground truth.  
Move the OCR data into the feed folder

In [14]:
def remove_box():
  input_folder = feed
  output_folder = output

  for filename in os.listdir(input_folder):
    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, filename)
    print(filename)
    with open(input_path, 'r') as infile, open(output_path, 'w') as outfile:
      for line in infile:
        try:
          line = line.strip()
          parts = line.split(',')
          if len(parts) > 8:
            # Rejoin the parts after the first 8 items
            new_line = ','.join(parts[8:])
            # Write the modified line to the output file
            outfile.write(new_line + '\n')
          else:
            # Write the line as it is if it doesn't have 8 numbers
            outfile.write(line)
        except UnicodeDecodeError as e:
          print(f"Error processing line: {line}")
          print(f"file: {filename}")
remove_box()


X00016469612.txt
X00016469619.txt
X00016469620.txt
X00016469622.txt
X00016469623.txt
X00016469669.txt
X00016469670.txt
X00016469671.txt
X00016469672.txt
X00016469676.txt
X51005200931.txt
X51005200938.txt
X51005230605.txt
X51005230616.txt
X51005230617.txt
X51005230621.txt
X51005230648.txt
X51005230657.txt
X51005230659.txt
X51005255805.txt
X51005268200.txt
X51005268262.txt
X51005268275.txt
X51005268400.txt
X51005268408.txt
X51005268472.txt
X51005288570.txt
X51005301659.txt
X51005301661.txt
X51005301666.txt
X51005301667.txt
X51005303661.txt
X51005306399.txt
X51005337867.txt
X51005337872.txt
X51005337877.txt
X51005361883.txt
X51005361895.txt
X51005361897.txt
X51005361898.txt
X51005361900.txt
X51005361906.txt
X51005361907.txt
X51005361908.txt
X51005361912.txt
X51005361923.txt
X51005361946.txt
X51005361950.txt
X51005365179.txt
X51005365187.txt
X51005433492.txt
X51005433494.txt
X51005433514.txt
X51005433518.txt
X51005433522.txt
X51005433533.txt
X51005433538.txt
X51005433541.txt
X51005433543.t

replace the entries in the feed folder with the entries in the output folder

In [16]:
def choose_50():

  to_write_to = output

  all_files = [f for f in os.listdir(feed) if os.path.isfile(os.path.join(feed, f))]
  selected_files = random.sample(all_files, 50)

  for file_name in selected_files:
    source_file_path = os.path.join(feed, file_name)
    destination_file_path = os.path.join(to_write_to, file_name)
    shutil.move(source_file_path, destination_file_path)

choose_50()

Move the entries in the output folder into the feed folder

In [21]:
def get_matching_pictures():
        
  txt_files = {os.path.splitext(f)[0] for f in os.listdir(feed) if os.path.isfile(os.path.join(feed, f)) and f.endswith('.txt')}
      
  for png_file in os.listdir(feed_pictures):
    print(png_file)
    if png_file.endswith('.jpg'):
        png_file_base = os.path.splitext(png_file)[0]
        if png_file_base in txt_files:
            # Copy the matching png file to the destination folder
            source_file_path = os.path.join(feed_pictures, png_file)
            destination_file_path = os.path.join(output, png_file)
            shutil.move(source_file_path, destination_file_path)
            print(f"Moved picture file {png_file} to output folder")
get_matching_pictures()

X00016469612.jpg
X00016469619.jpg
X00016469620.jpg
X00016469622.jpg
X00016469623.jpg
X00016469669.jpg
X00016469670.jpg
X00016469671.jpg
X00016469672.jpg
X00016469676.jpg
X51005200931.jpg
X51005200938.jpg
X51005230605.jpg
X51005230616.jpg
X51005230617.jpg
X51005230621.jpg
X51005230648.jpg
X51005230657.jpg
X51005230659.jpg
X51005255805.jpg
X51005268200.jpg
X51005268262.jpg
X51005268275.jpg
X51005268400.jpg
X51005268408.jpg
X51005268472.jpg
X51005288570.jpg
X51005301659.jpg
X51005301661.jpg
X51005301666.jpg
X51005301667.jpg
X51005303661.jpg
X51005306399.jpg
X51005337867.jpg
X51005337872.jpg
X51005337877.jpg
X51005361883.jpg
X51005361895.jpg
X51005361897.jpg
X51005361898.jpg
X51005361900.jpg
X51005361906.jpg
X51005361907.jpg
X51005361908.jpg
X51005361912.jpg
X51005361923.jpg
X51005361946.jpg
X51005361950.jpg
X51005365179.jpg
X51005365187.jpg
X51005433492.jpg
X51005433494.jpg
X51005433514.jpg
X51005433518.jpg
X51005433522.jpg
X51005433533.jpg
X51005433538.jpg
X51005433541.jpg
X51005433543.j

In [ ]:
#Get Matching LATIN

In [ ]:
def load_and_check_documents():
    directory_path = output
    # Iterate over all files in the specified directory
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        
        # Ensure we're working with a file, not a subdirectory
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                text_content = file.read()
            
            # Use regex to extract the dictionary-like content
            match = re.search(r'\{.*\}', text_content, re.DOTALL)
            if match:
                dict_string = match.group(0)
                try:
                    # Safely evaluate the string to a dictionary
                    data_dict = ast.literal_eval(dict_string)
                    
                    # Check if the dictionary has exactly 4 keys
                    if len(data_dict) != 4:
                        print(f"File '{filename}' does not have 4 keys. It has {len(data_dict)} keys.")
                except Exception as e:
                    print(f"Error processing file '{filename}': {e}")
            else:
                print(f"File '{filename}' does not contain a valid dictionary.")